In [72]:
import numpy as np
import pandas as pd

In [73]:
with open("hashcode.in") as f:
    D, I, S, V, F = tuple(int(temp) for temp in f.readline().split())
    print(D)
    street_info = {}
    for temp in range(S):
        street_info[temp]=tuple(temp for temp in f.readline().split())
    car_info = {}
    for temp in range(V):
        car_info[temp]=tuple(temp for temp in f.readline().split())

street_info = pd.DataFrame.from_dict(street_info, orient = 'index',columns = ['start_intersection','end_intersection','street_name','travel_time']).set_index('street_name')
car_info = pd.DataFrame.from_dict(car_info, orient = 'index',columns = ['n_streets'] + ['street_'+ str(i) for i in range(120)])

7071


In [74]:
start_intersection = street_info.loc[:,'start_intersection']
end_intersection = street_info.loc[:,'end_intersection']

In [75]:
street_info.groupby('start_intersection').agg('count')['end_intersection']

start_intersection
0       108
1       140
10      212
100      21
1000      9
       ... 
995      13
996      13
997      24
998       9
999      12
Name: end_intersection, Length: 8000, dtype: int64

In [76]:
start_intersection['a-e']

'0'

In [77]:
car_info.iloc[:,1:].apply(lambda col: col.apply(lambda v: start_intersection[v]),axis= 0)

,street_0,street_1,street_2,street_3,street_4,street_5,street_6,street_7,street_8,street_9,...,street_110,street_111,street_112,street_113,street_114,street_115,street_116,street_117,street_118,street_119
0,5853,1412,65,1007,3970,6022,8,61,3954,748,...,1597,4709,3738,623,95,11,5803,1258,57,1656
1,2978,32,550,261,2222,7679,6073,4763,5666,426,...,4517,64,2712,1314,609,1274,6,3226,5365,10
2,6994,479,58,1260,28,3673,4299,4488,1158,1417,...,162,4966,240,2194,8,1288,1252,896,621,4570
3,7663,4451,1671,6514,7768,4,5322,1576,5808,5890,...,24,4806,440,621,896,3609,103,4307,4091,124
4,3947,7026,1166,262,3548,1181,6103,882,218,1420,...,1461,4820,5411,3607,5984,1469,63,7525,151,4636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3535,2231,4947,39,3384,5430,7361,1322,3659,246,...,2273,5597,6219,4386,98,2184,1311,758,71,3239
996,760,1162,7247,8,471,7149,1,210,535,7,...,5656,2110,518,2031,2821,685,3074,1475,190,5756
997,3793,3456,564,4945,127,2629,38,1205,1154,1408,...,1256,5989,120,1565,1487,650,6413,3421,3504,4927
998,7259,12,2371,213,7651,7169,62,4549,1059,1252,...,41,5,4248,3434,2617,6890,4908,5678,1858,74


In [80]:
street_info['end_light_schedule'] = np.ones(street_info.shape[0],).astype('int')
print(street_info.shape)
street_info.head()

(63968, 4)


,start_intersection,end_intersection,travel_time,end_light_schedule
street_name,,,,
a-e,0,4,40,1
e-a,4,0,40,1
a-g,0,6,35,1
g-a,6,0,35,1
a-i,0,8,46,1


In [122]:
traffic_light_info = pd.DataFrame(street_info.groupby('end_intersection').apply(lambda x: list(x.index)), columns = ['entering_from'])
traffic_light_info['exit_to'] = street_info.groupby('start_intersection').apply(lambda x: list(x.index))
traffic_light_info['schedule'] = traffic_light_info['entering_from'].apply(lambda lst: [1 for i in lst])
traffic_light_info['schedule_pairs'] = traffic_light_info.apply(lambda row: dict(zip(row['entering_from'],row['schedule'])), axis = 1)

traffic_light_info.head()

,entering_from,exit_to,schedule,schedule_pairs
end_intersection,,,,
0,"[e-a, g-a, i-a, bi-a, bj-a, cb-a, da-a, df-a, ...","[a-e, a-g, a-i, a-bi, a-bj, a-cb, a-da, a-df, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","{'e-a': 1, 'g-a': 1, 'i-a': 1, 'bi-a': 1, 'bj-..."
1,"[e-b, f-b, bb-b, bc-b, bi-b, df-b, dh-b, ef-b,...","[b-e, b-f, b-bb, b-bc, b-bi, b-df, b-dh, b-ef,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","{'e-b': 1, 'f-b': 1, 'bb-b': 1, 'bc-b': 1, 'bi..."
10,"[c-ba, d-ba, f-ba, i-ba, bc-ba, bf-ba, cc-ba, ...","[ba-c, ba-d, ba-f, ba-i, ba-bc, ba-bf, ba-cc, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","{'c-ba': 1, 'd-ba': 1, 'f-ba': 1, 'i-ba': 1, '..."
100,"[bi-baa, gj-baa, id-baa, jh-baa, bif-baa, dha-...","[baa-bi, baa-gj, baa-id, baa-jh, baa-bif, baa-...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","{'bi-baa': 1, 'gj-baa': 1, 'id-baa': 1, 'jh-ba..."
1000,"[jf-baaa, baj-baaa, eii-baaa, ejh-baaa, cbih-b...","[baaa-jf, baaa-baj, baaa-eii, baaa-ejh, baaa-c...","[1, 1, 1, 1, 1, 1, 1, 1, 1]","{'jf-baaa': 1, 'baj-baaa': 1, 'eii-baaa': 1, '..."


In [136]:
class traffic_light:
    def __init__(self, traffic_light_id = None, entering_roads = [], exiting_roads = [], schedule_pairs = {}):
        self.traffic_light_id = traffic_light_id
        self.entering_roads = entering_roads
        self.exiting_roads = exiting_roads
        self.schedule_pairs = schedule_pairs #a dict
        self.schedule_queue = []
    def initialize_as_one(self):
        for road in self.entering_roads:
            self.schedule_pairs[road] = 1
            self.schedule_queue.append(road)
    def initialize(self):
        for road, elapse in self.schedule_pairs.items():
            self.schedule_queue.extend([road]*elapse)
    def sprint(self):
        #print(self.traffic_light_id)
        #print(self.entering_roads)
        #print(self.exiting_roads)
        #print(self.schedule_pairs)
        print(self.schedule_queue)
    def passTime(self):
        temp = self.schedule_queue[0]
        self.schedule_queue = self.schedule_queue[1:] + [temp]
    def isGreenLight(self, enter_from = None):
        if enter_from is None:
            isGreenLight = None
        else:
            isGreenLight = enter_from == self.schedule_quene[0]
        self.passTime()
        return isGreenLight
t = traffic_light('0',traffic_light_info.loc['0','entering_from'], traffic_light_info.loc['0','exit_to'], traffic_light_info.loc['0','schedule_pairs'])
t.initialize()
t.isGreenLight()

In [124]:
class car:
    def __init__(self, car_id = None, path = [], path_pairs=[]):
        self.car_id = car_id
        self.path = path
        self.current_position = None
        self.isWaiting = False
        self.path_pairs = path_pairs
        self.travel_queue = []
    def initialize(self):
        for road, elapse in self.path_pairs:
            self.travel_queue.extend([None] * (elapse-1) + [road])
    def sprint(self):
        print(self.travel_queue)
    def passCheck(self, isTraveling = True): #WIP car time pass
        if not isTraveling:
            return None
        if travel_queue = []:
            return 'Finished'
        else:
            position = travel_queue[0]
            self.travel_queue = self.travel_queue[1:]
            if position is not None:
                return position
            else:
                return None
    def isGreenLight(self, enter_from = None):
        if enter_from is None:
            isGreenLight = None
        else:
            isGreenLight = enter_from == self.schedule_quene[0]
        self.passTime()
        return isGreenLight


['e-a',
 'g-a',
 'i-a',
 'bi-a',
 'bj-a',
 'cb-a',
 'da-a',
 'df-a',
 'di-a',
 'eb-a',
 'gd-a',
 'id-a',
 'ij-a',
 'jf-a',
 'bah-a',
 'bdc-a',
 'bea-a',
 'bfe-a',
 'bhi-a',
 'ccj-a',
 'cje-a',
 'cji-a',
 'dce-a',
 'dch-a',
 'dda-a',
 'dga-a',
 'edj-a',
 'ehi-a',
 'fgb-a',
 'fha-a',
 'fjb-a',
 'hcd-a',
 'heg-a',
 'iej-a',
 'ihb-a',
 'jfi-a',
 'baac-a',
 'babd-a',
 'bcfc-a',
 'bdfi-a',
 'bdhf-a',
 'becd-a',
 'befg-a',
 'bfaf-a',
 'bgbh-a',
 'bjie-a',
 'cagi-a',
 'cchi-a',
 'ccie-a',
 'cdha-a',
 'ceae-a',
 'cehe-a',
 'cehf-a',
 'cgae-a',
 'cgdf-a',
 'chad-a',
 'chbe-a',
 'ched-a',
 'chhb-a',
 'cjef-a',
 'dafd-a',
 'dcdf-a',
 'ddbf-a',
 'dddi-a',
 'ddfb-a',
 'ddhe-a',
 'dfbh-a',
 'dgga-a',
 'dggg-a',
 'dghf-a',
 'didd-a',
 'djga-a',
 'eaai-a',
 'eafe-a',
 'ebhb-a',
 'ecij-a',
 'ecje-a',
 'eeag-a',
 'eebh-a',
 'efba-a',
 'efcd-a',
 'ehjh-a',
 'eice-a',
 'eiea-a',
 'ejjf-a',
 'fade-a',
 'fajf-a',
 'fcci-a',
 'fffa-a',
 'fjef-a',
 'gafd-a',
 'gbha-a',
 'gcfe-a',
 'gdjh-a',
 'geab-a',
 'geci-a